# Práctica 1 - Naive Bayes

_Pareja 6_
* David Kaack Sánchez
* Carlos César Rodríguez García

## Setup

### Importaciones

In [1]:
from Clasificador import Clasificador, ClasificadorNaiveBayes, ClasificadorNaiveBayesScikit
from Datos import Datos
from EstrategiaParticionado import ValidacionCruzada, ValidacionSimple

import copy
import pandas as pd
from sklearn.naive_bayes import CategoricalNB, GaussianNB, MultinomialNB

### Estrategia de particionado

In [2]:
validacion_cruzada = ValidacionCruzada(numeroParticiones=5)
validacion_simple = ValidacionSimple(numeroEjecuciones=5, proporcionTest=30)

### Clasificador generico

In [3]:
clasificador = Clasificador() # Solo se usa para correr clasficador.validacion()

### Utilerías

Funciones que ayudan al procesado de datos y la creación de tablas para analizarlos.

In [4]:
def _validacion_naive_bayes_propio(datos, estrategia_particionado, con_laplace):
    estrategia_particionado = copy.deepcopy(estrategia_particionado)
    return clasificador.validacion(estrategia_particionado, datos, ClasificadorNaiveBayes(con_laplace=con_laplace))

def validacion_naive_bayes_propio(dataset):
    datos = Datos(f"{dataset}.csv")
    
    resultados_vc = _validacion_naive_bayes_propio(datos, validacion_cruzada, con_laplace=False)
    resultados_vc_cl = _validacion_naive_bayes_propio(datos, validacion_cruzada, con_laplace=True)
    
    resultados_vs = _validacion_naive_bayes_propio(datos, validacion_simple, con_laplace=False)
    resultados_vs_cl = _validacion_naive_bayes_propio(datos, validacion_simple, con_laplace=True)

    columnas = [
        "Conjunto de Datos",
        "Estrategia Particionado",
        "Corrección de Laplace",
        "Error Promedio",
        "Desviación Estándar",
    ]

    filas = [
        (
            dataset,
            "Validación Cruzada",
            False,
            resultados_vc[0],
            resultados_vc[1],
        ),
        (
            dataset,
            "Validación Cruzada",
            True,
            resultados_vc_cl[0],
            resultados_vc_cl[1],
        ),
        (
            dataset,
            "Validación Simple",
            False,
            resultados_vs[0],
            resultados_vs[1],
        ),
        (
            dataset,
            "Validación Simple",
            True,
            resultados_vs_cl[0],
            resultados_vs_cl[1],
        )
    ]

    return pd.DataFrame(filas, columns=columnas)

def _validacion_naive_bayes_scikit(datos, estrategia_particionado, instancia_clasificador, con_transformacion):
    estrategia_particionado = copy.deepcopy(estrategia_particionado)
    return clasificador.validacion(estrategia_particionado, datos, ClasificadorNaiveBayesScikit(instancia_clasificador, con_transformacion=con_transformacion))

def validacion_naive_bayes_scikit(dataset, clasificador):
    datos = Datos(f"{dataset}.csv")

    instancia_clasificador = clasificador(fit_prior=True) if not isinstance(clasificador(), GaussianNB) else clasificador()
    instancia_clasificador_cl = clasificador(fit_prior=True, alpha=1) if not isinstance(clasificador(), GaussianNB) else clasificador()

    resultados_vc = _validacion_naive_bayes_scikit(datos, validacion_cruzada, instancia_clasificador, con_transformacion=False)
    resultados_vc_cl = _validacion_naive_bayes_scikit(datos, validacion_cruzada, instancia_clasificador_cl, con_transformacion=False)
    resultados_vc_ohe = _validacion_naive_bayes_scikit(datos, validacion_cruzada, instancia_clasificador, con_transformacion=True)
    resultados_vc_cl_ohe = _validacion_naive_bayes_scikit(datos, validacion_cruzada, instancia_clasificador_cl, con_transformacion=True)

    
    resultados_vs = _validacion_naive_bayes_scikit(datos, validacion_simple, instancia_clasificador, con_transformacion=False)
    resultados_vs_cl = _validacion_naive_bayes_scikit(datos, validacion_simple, instancia_clasificador_cl, con_transformacion=False)
    resultados_vs_ohe = _validacion_naive_bayes_scikit(datos, validacion_simple, instancia_clasificador, con_transformacion=True)
    resultados_vs_cl_ohe = _validacion_naive_bayes_scikit(datos, validacion_simple, instancia_clasificador_cl, con_transformacion=True)

    columnas = [
        "Conjunto de Datos",
        "Estrategia Particionado",
        "Corrección de Laplace",
        "One Hot Encoding",
        "Error Promedio",
        "Desviación Estándar",
    ]

    filas = [
        (
            dataset,
            "Validación Cruzada",
            False,
            False,
            resultados_vc[0],
            resultados_vc[1],
        ),
        (
            dataset,
            "Validación Cruzada",
            True,
            False,
            resultados_vc_cl[0],
            resultados_vc_cl[1],
        ),
        (
            dataset,
            "Validación Cruzada",
            False,
            True,
            resultados_vc_ohe[0],
            resultados_vc_ohe[1],
        ),
        (
            dataset,
            "Validación Cruzada",
            True,
            True,
            resultados_vc_cl_ohe[0],
            resultados_vc_cl_ohe[1],
        ),
        (
            dataset,
            "Validación Simple",
            False,
            False,
            resultados_vs[0],
            resultados_vs[1],
        ),
        (
            dataset,
            "Validación Simple",
            True,
            False,
            resultados_vs_cl[0],
            resultados_vs_cl[1],
        ),
        (
            dataset,
            "Validación Simple",
            False,
            True,
            resultados_vs_ohe[0],
            resultados_vs_ohe[1],
        ),
        (
            dataset,
            "Validación Simple",
            True,
            True,
            resultados_vs_cl_ohe[0],
            resultados_vs_cl_ohe[1],
        ),
    ]

    return pd.DataFrame(filas, columns=columnas)

## Apartado 1 - Naive Bayes Propio

### Entrenamiento y clasificación para dataset __heart__

In [5]:
resultados_heart = validacion_naive_bayes_propio("heart")

### Entrenamiento y clasificación para dataset __tic-tac-toe__

In [6]:
resultados_tic_tac_toe = validacion_naive_bayes_propio("tic-tac-toe")

### Analisis de resultados

In [7]:
resultados_naive_bayes_propio = pd.concat([
    resultados_heart.sort_values(by="Error Promedio"),
    resultados_tic_tac_toe.sort_values(by="Error Promedio")
])
resultados_naive_bayes_propio

,Conjunto de Datos,Estrategia Particionado,Corrección de Laplace,Error Promedio,Desviación Estándar
2,heart,Validación Simple,False,0.140364,0.019048
3,heart,Validación Simple,True,0.141091,0.019704
0,heart,Validación Cruzada,False,0.143811,0.020987
1,heart,Validación Cruzada,True,0.143811,0.020987
1,tic-tac-toe,Validación Cruzada,True,0.292343,0.029646
0,tic-tac-toe,Validación Cruzada,False,0.293385,0.028196
2,tic-tac-toe,Validación Simple,False,0.305226,0.025566
3,tic-tac-toe,Validación Simple,True,0.305226,0.025566


El error promedio en todos los casos es menor al 30%, lo cual consideramos un rendimiento regular (es mejor que una decisión al azar). Lo interesante de estos datos es la reducción de la desviación estándar con el datased __heart__ cuando se usa validación simple como estrategia de particionado. En conjuntos de datos pequeños, la validación cruzada puede tener una desviación estándar más alta debido a la limitada cantidad de datos para realizar las divisiones. La validación simple puede ser más estable en tales casos.

En general, el modelo tiene resultados estables para ambas estrategias de particionado.

#### Efectos de Corrección de Laplace

Observamos que la corrección de Laplace tiene un impacto limitado en estos escenarios. De hecho, en la mayoría de los casos, conduce a un aumento en el error. Esta situación puede atribuirse a la falta de atributos categóricos en los cuales aplicar esta corrección, o a la presencia de un número reducido de ellos; aunque en este caso, probablemente se deba a una buena distribución de las categorías en los conjuntos de datos analizados.

## Apartado 2 - Naive Bayes Scikit-learn

### Entrenamiento y clasificación para dataset __tic-tac-toe__

#### CategoricalNB
Para este clasficidor, se asume que calcula las probabilidades a priori (`fit_prior=True`) y se evalúa tanto para el caso con corrección de Laplace y sin ella (`alpha=1`).

In [8]:
resultados_tic_tac_toe_categorical_nb = validacion_naive_bayes_scikit("tic-tac-toe", CategoricalNB)
resultados_tic_tac_toe_categorical_nb

,Conjunto de Datos,Estrategia Particionado,Corrección de Laplace,One Hot Encoding,Error Promedio,Desviación Estándar
0,tic-tac-toe,Validación Cruzada,False,False,0.292343,0.029646
1,tic-tac-toe,Validación Cruzada,True,False,0.292343,0.029646
2,tic-tac-toe,Validación Cruzada,False,True,0.304876,0.033982
3,tic-tac-toe,Validación Cruzada,True,True,0.304876,0.033982
4,tic-tac-toe,Validación Simple,False,False,0.304530,0.026315
5,tic-tac-toe,Validación Simple,True,False,0.304530,0.026315
6,tic-tac-toe,Validación Simple,False,True,0.319861,0.027025
7,tic-tac-toe,Validación Simple,True,True,0.319861,0.027025


#### GaussianNB
Este clasificador asume que todas las variables son continuas y distribuidas normalmente. Por esta razón, no se puede evaluar el la corrección de Laplace, pues no tiene ningún efecto.

In [9]:
resultados_tic_tac_toe_gaussian_nb = validacion_naive_bayes_scikit("tic-tac-toe", GaussianNB)
resultados_tic_tac_toe_gaussian_nb = resultados_tic_tac_toe_gaussian_nb[resultados_tic_tac_toe_gaussian_nb["Corrección de Laplace"] == False]
resultados_tic_tac_toe_gaussian_nb

,Conjunto de Datos,Estrategia Particionado,Corrección de Laplace,One Hot Encoding,Error Promedio,Desviación Estándar
0,tic-tac-toe,Validación Cruzada,False,False,0.276674,0.024452
2,tic-tac-toe,Validación Cruzada,False,True,0.315320,0.033065
4,tic-tac-toe,Validación Simple,False,False,0.278746,0.031552
6,tic-tac-toe,Validación Simple,False,True,0.331707,0.040836


#### MultinomialNB
Funciona mejor para variables discretas, al igual que CategoricalNB. De igual modo, se asume que calcula las probabilidades a priori (`fit_prior=True`) y se evalúa tanto para el caso con corrección de Laplace y sin ella (`alpha=1`).

In [10]:
resultados_tic_tac_toe_multinomial_nb = validacion_naive_bayes_scikit("tic-tac-toe", MultinomialNB)
resultados_tic_tac_toe_multinomial_nb

,Conjunto de Datos,Estrategia Particionado,Corrección de Laplace,One Hot Encoding,Error Promedio,Desviación Estándar
0,tic-tac-toe,Validación Cruzada,False,False,0.341383,0.027139
1,tic-tac-toe,Validación Cruzada,True,False,0.341383,0.027139
2,tic-tac-toe,Validación Cruzada,False,True,0.292343,0.029646
3,tic-tac-toe,Validación Cruzada,True,True,0.292343,0.029646
4,tic-tac-toe,Validación Simple,False,False,0.326132,0.037759
5,tic-tac-toe,Validación Simple,True,False,0.326132,0.037759
6,tic-tac-toe,Validación Simple,False,True,0.304530,0.026315
7,tic-tac-toe,Validación Simple,True,True,0.304530,0.026315


### Comparación de clasificadores

In [11]:
# Creación de columna identificadora de clasificadores
resultados_tic_tac_toe_categorical_nb["Clasificador"] = "CategoricalNB"
resultados_tic_tac_toe_gaussian_nb["Clasificador"] = "GaussianNB"
resultados_tic_tac_toe_multinomial_nb["Clasificador"] = "MultinomialNB"

resultados_tic_tac_toe_scikit = pd.concat([
    resultados_tic_tac_toe_categorical_nb,
    resultados_tic_tac_toe_gaussian_nb,
    resultados_tic_tac_toe_multinomial_nb
])
resultados_tic_tac_toe_scikit = resultados_tic_tac_toe_scikit.reset_index(drop=True)

resultados_tic_tac_toe_scikit.sort_values(by="Error Promedio")

,Conjunto de Datos,Estrategia Particionado,Corrección de Laplace,One Hot Encoding,Error Promedio,Desviación Estándar,Clasificador
8,tic-tac-toe,Validación Cruzada,False,False,0.276674,0.024452,GaussianNB
10,tic-tac-toe,Validación Simple,False,False,0.278746,0.031552,GaussianNB
0,tic-tac-toe,Validación Cruzada,False,False,0.292343,0.029646,CategoricalNB
1,tic-tac-toe,Validación Cruzada,True,False,0.292343,0.029646,CategoricalNB
15,tic-tac-toe,Validación Cruzada,True,True,0.292343,0.029646,MultinomialNB
14,tic-tac-toe,Validación Cruzada,False,True,0.292343,0.029646,MultinomialNB
18,tic-tac-toe,Validación Simple,False,True,0.304530,0.026315,MultinomialNB
19,tic-tac-toe,Validación Simple,True,True,0.304530,0.026315,MultinomialNB
5,tic-tac-toe,Validación Simple,True,False,0.304530,0.026315,CategoricalNB
4,tic-tac-toe,Validación Simple,False,False,0.304530,0.026315,CategoricalNB


Estos son resultados inesperados. `tic-tac-toe` cuenta con una gran presencia de atributos nominales, por lo que es contrauitivo ver a GaussianNB como el mejor modelo para este conjunto de datos (tanto en error promedio como desviación estándar). Inclusive, la configuración ganadora (sin One Hot Encoding) estaría asumiendo las variables nominales como números secuenciales en lugar de categorías.

En general el rango de los errores promedio es muy reducido entre 0.27 y 0.34, lo cual demuestra comportamientos apropiados para cada modelo.

#### Efecto de One Hot Encoding (OHE)

In [12]:
filas_con_ohe = resultados_tic_tac_toe_scikit[resultados_tic_tac_toe_scikit["One Hot Encoding"] == True]
error_promedio = filas_con_ohe["Error Promedio"].mean()
desviacion_estandar = filas_con_ohe["Desviación Estándar"].mean()

print(f"Los casos donde se usó OHE tienen un error promedio de {error_promedio} y una desviación estándar de {desviacion_estandar}")

Los casos donde se usó OHE tienen un error promedio de 0.3090244567530753 y una desviación estándar de 0.030783981239946086


In [13]:
filas_sin_ohe = resultados_tic_tac_toe_scikit[resultados_tic_tac_toe_scikit["One Hot Encoding"] == False]
error_promedio = filas_sin_ohe["Error Promedio"].mean()
desviacion_estandar = filas_sin_ohe["Desviación Estándar"].mean()

print(f"Los casos donde NO se usó OHE tienen un error promedio de {error_promedio} y una desviación estándar de {desviacion_estandar}")

Los casos donde NO se usó OHE tienen un error promedio de 0.3084195995311673 y una desviación estándar de 0.029772333093957014


Se observan resultados notoriamente similares en términos generales, independientemente de si se utiliza el One-Hot Encoding (OHE) o no. Sin embargo, para el clasificador MultinomialNB, se aprecia un rendimiento superior al emplear OHE. En este caso, los errores promedio con OHE se mantienen consistentemente por debajo del 0.30, mientras que sin OHE, tienden a estar por encima de este valor. Esto se debe al hecho de que el MultinomialNB está diseñado para manejar variables categóricas, y el OHE facilita que las representaciones de atributos nominales se interpreten como clases distintas, asignando a cada categoría su propia columna binaria.

Esta ventaja es aún más evidente debido a la naturaleza del conjunto de datos de tic-tac-toe, que, como se mencionó previamente, contiene una considerable cantidad de atributos categóricos.

### Entrenamiento y clasificación para dataset __heart__

In [14]:
resultados_heart_scikit = validacion_naive_bayes_scikit("heart", GaussianNB)
resultados_heart_scikit = resultados_heart_scikit[resultados_heart_scikit["Corrección de Laplace"] == False]
resultados_heart_scikit

,Conjunto de Datos,Estrategia Particionado,Corrección de Laplace,One Hot Encoding,Error Promedio,Desviación Estándar
0,heart,Validación Cruzada,False,False,0.142730,0.026703
2,heart,Validación Cruzada,False,True,0.137259,0.019331
4,heart,Validación Simple,False,False,0.153455,0.023044
6,heart,Validación Simple,False,True,0.138909,0.019569


#### ¿Por qué crees que no utilizamos los otros dos algoritmos aquí? ¿qué transformación/es podríamos hacer para resolver el problema?
Porque este dataset contiene atributos continuos. Para resolver el problema se pueden escalar estos atributos por ejemplo con la clase `StandardScaler` de sklearn. Así se transforma el atributo de tipo continuo a un tipo multinominal

#### Efecto de One Hot Encoding (OHE)

Aquí el impacto es mucho más evidente que con el dataset `tic-tac-toe`. Debido a que GaussianNB asume a todas las variables como continuas y normalmente distribuidas, habría interpretaciones incorrectas de las variables nominales que hemos transformado a número (se tomarían como secuencias y no categorías). OHE ayuda a resolver este problema y los resultados llegan a superar nuestra propia implementación de Naive Bayes.

## Apartado 3 - Conclusión

Se aprecian resultados notoriamente similares entre nuestra implementación y la biblioteca Scikit-Learn. Esto es un indicativo alentador de la calidad de nuestro trabajo. En varios de los casos, parece que nuestro clasificador personalizado supera en precisión a las implementaciones de la librería. Esta diferencia se atribuye al enfoque específico que hemos adoptado, donde consideramos la naturaleza de cada atributo, ya sea categórico (utilizando probabilidades) o numérico (aplicando el modelo Gaussiano). En contraste, los modelos de Scikit-Learn manejan los datos de una manera más general, tratando todos los atributos como continuos o categóricos por igual.

A pesar de esta diferencia en el manejo de atributos, los algoritmos de Scikit-Learn presentan un rendimiento destacado. Esto podría deberse, en parte, a las optimizaciones internas incorporadas en la biblioteca.

#### Comparación dataset __tic-tac-toe__

In [15]:
# Mejor resultado de Naive Bayes Propio
resultado_naive_bayes_propio_tic_tac_toe = resultados_naive_bayes_propio[resultados_naive_bayes_propio["Conjunto de Datos"] == "tic-tac-toe"]
resultado_naive_bayes_propio_tic_tac_toe.loc[resultado_naive_bayes_propio_tic_tac_toe["Error Promedio"].idxmin()]

Conjunto de Datos                 tic-tac-toe
Estrategia Particionado    Validación Cruzada
Corrección de Laplace                    True
Error Promedio                       0.292343
Desviación Estándar                  0.029646
Name: 1, dtype: object

In [16]:
# Mejor resultado de Naive Bayes Scikit
resultados_tic_tac_toe_scikit.loc[resultados_tic_tac_toe_scikit["Error Promedio"].idxmin()]

Conjunto de Datos                 tic-tac-toe
Estrategia Particionado    Validación Cruzada
Corrección de Laplace                   False
One Hot Encoding                        False
Error Promedio                       0.276674
Desviación Estándar                  0.024452
Clasificador                       GaussianNB
Name: 8, dtype: object

_Ganador con base a mejor resultado:_ Clasificador Scikit

In [17]:
# Peor resultado de Naive Bayes Propio
resultado_naive_bayes_propio_tic_tac_toe.loc[resultado_naive_bayes_propio_tic_tac_toe["Error Promedio"].idxmax()]

Conjunto de Datos                tic-tac-toe
Estrategia Particionado    Validación Simple
Corrección de Laplace                  False
Error Promedio                      0.305226
Desviación Estándar                 0.025566
Name: 2, dtype: object

In [18]:
# Peor resultado de Naive Bayes Scikit
resultados_tic_tac_toe_scikit.loc[resultados_tic_tac_toe_scikit["Error Promedio"].idxmax()]

Conjunto de Datos                 tic-tac-toe
Estrategia Particionado    Validación Cruzada
Corrección de Laplace                   False
One Hot Encoding                        False
Error Promedio                       0.341383
Desviación Estándar                  0.027139
Clasificador                    MultinomialNB
Name: 12, dtype: object

_Ganador con base a peor resultado:_ Clasificador propio

#### Comparación dataset __heart__

In [19]:
# Mejor resultado de Naive Bayes Propio
resultado_naive_bayes_propio_heart = resultados_naive_bayes_propio[resultados_naive_bayes_propio["Conjunto de Datos"] == "heart"]
resultado_naive_bayes_propio_heart.loc[resultado_naive_bayes_propio_heart["Error Promedio"].idxmin()]

Conjunto de Datos                      heart
Estrategia Particionado    Validación Simple
Corrección de Laplace                  False
Error Promedio                      0.140364
Desviación Estándar                 0.019048
Name: 2, dtype: object

In [20]:
# Mejor resultado de Naive Bayes Scikit
resultados_heart_scikit.loc[resultados_heart_scikit["Error Promedio"].idxmin()]

Conjunto de Datos                       heart
Estrategia Particionado    Validación Cruzada
Corrección de Laplace                   False
One Hot Encoding                         True
Error Promedio                       0.137259
Desviación Estándar                  0.019331
Name: 2, dtype: object

_Ganador con base a mejor resultado_: Clasificador Scikit

In [21]:
# Peor resultado de Naive Bayes Propio
resultado_naive_bayes_propio_heart.loc[resultado_naive_bayes_propio_heart["Error Promedio"].idxmax()]

Conjunto de Datos                       heart
Estrategia Particionado    Validación Cruzada
Corrección de Laplace                   False
Error Promedio                       0.143811
Desviación Estándar                  0.020987
Name: 0, dtype: object

In [22]:
# Peor resultado de Naive Bayes Scikit
resultados_heart_scikit.loc[resultados_heart_scikit["Error Promedio"].idxmax()]

Conjunto de Datos                      heart
Estrategia Particionado    Validación Simple
Corrección de Laplace                  False
One Hot Encoding                       False
Error Promedio                      0.153455
Desviación Estándar                 0.023044
Name: 4, dtype: object

_Ganador con base a peor resultado_: Clasificador propio